# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LORA
* Model: gpt2
* Evaluation approach: accuracy using the trainer.evaluate()
* Fine-tuning dataset: financial_phrasebank - sentences_allagree

In [1]:
pip install --upgrade -q peft

Note: you may need to restart the kernel to use updated packages.


In [2]:
model_name = 'gpt2' #'bert-base-uncased'#
dataset = "financial_phrasebank"
subset = 'sentences_allagree'

padding_token = '[PAD]'

epochs = 5

In [3]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

per_device_train_batch_size = torch.cuda.device_count()
if per_device_train_batch_size == 0:
    per_device_train_batch_size = 1

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [4]:
## Load Dataset

In [5]:
from datasets import load_dataset

splits = ["train", "val", "test"]

ds = load_dataset(path = dataset, name = subset, split='train').shuffle(seed = 42)
ds = ds.train_test_split(test_size = 0.4, stratify_by_column = 'label', seed = 42)
temp = ds['test'].train_test_split(test_size = 0.3, stratify_by_column = 'label', seed = 4)

ds['val'] = temp['train']
ds['test'] = temp['test']
ds = ds.with_format("torch")
ds

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 1358
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 272
    })
    val: Dataset({
        features: ['sentence', 'label'],
        num_rows: 634
    })
})

In [6]:
## Define Model and Tokenizer

In [7]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer


names = ds['train'].features['label'].names
id2label = {index: item for index, item in enumerate(names)}
label2id = {item: index for index, item in enumerate(names)}

# load model
foundation_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    id2label=id2label,
    label2id=label2id,
    num_labels=len(names),

).to(device)


for param in foundation_model.parameters():
    param.requires_grad = True

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

max_length = tokenizer.model_max_length

if model_name == 'gpt2':
    tokenizer.pad_token = tokenizer.eos_token
    foundation_model.config.pad_token_id =  tokenizer.pad_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
## Tokenize Data

In [9]:
def tokenize_function(examples):
    
    outputs = tokenizer(examples["sentence"], padding="max_length",\
                                          truncation=True, return_tensors='pt',\
                                        max_length = max_length)
    return outputs

tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = ds[split].map(
        tokenize_function,
        batched=True,
        remove_columns=['sentence']
      )   


In [10]:
## Evalute Model

In [11]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

In [12]:
attention_mask = tokenized_dataset['test']['attention_mask'][:30].to(device)
input_ids = tokenized_dataset['test']['input_ids'][:30].to(device)
labels = tokenized_dataset['test']['label'][:30].to(device)
with torch.no_grad():
    output = foundation_model(input_ids, attention_mask=attention_mask)

torch.mean(output.logits.argmax(dim=1).eq(labels).to(torch.float32)).item()

0.3333333432674408

In [13]:
del attention_mask
del input_ids
del labels
del output
torch.cuda.empty_cache()

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [14]:
print(foundation_model)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=3, bias=False)
)


In [15]:
from peft import LoraConfig

if model_name == 'gpt2':
    target_modules=["c_proj"]
elif model_name == 'bert-base-uncased':
    target_modules=["query","key","value"]



lora_config = LoraConfig(
    r=1,
    lora_alpha=16, 
    target_modules=target_modules,
    lora_dropout=0.1, 
    bias="none",
    task_type="SEQ_CLS"
)

from peft import get_peft_model
peft_model = get_peft_model(foundation_model, lora_config)
print(peft_model.print_trainable_parameters())

trainable params: 66,816 || all params: 124,508,928 || trainable%: 0.05366362161595352
None


/home/student/.local/lib/python3.10/site-packages/peft/tuners/lora/layer.py:711: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [16]:
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

training_args = TrainingArguments(output_dir="./peft_project", 
                                  logging_steps = 10,
                                  num_train_epochs=epochs, 
                                  learning_rate= 2e-3,
                                  per_device_train_batch_size = per_device_train_batch_size, 
                                  per_device_eval_batch_size = per_device_train_batch_size, 
                                  evaluation_strategy="epoch",
                                  save_strategy='epoch',
                                  weight_decay=0.01,
                                  load_best_model_at_end=True)

trainer = Trainer(model=peft_model, 
                  args=training_args, 
                  train_dataset = tokenized_dataset['train'],                 # train_tokenized_datasets, 
                  eval_dataset =  tokenized_dataset['val'],                # val_tokenized_datasets,
                  tokenizer=tokenizer,
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=compute_metrics
)

In [17]:
print("Evaluating before training...")
print("Pre-training evaluation results:", trainer.evaluate())

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Evaluating before training...


Pre-training evaluation results: {'eval_loss': 3.422431707382202, 'eval_accuracy': 0.250788643533123, 'eval_runtime': 57.616, 'eval_samples_per_second': 11.004, 'eval_steps_per_second': 11.004}


In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.287900,1.387829,0.599369
2,1.145800,1.503470,0.747634
3,0.224600,1.330181,0.752366
4,0.030800,0.963500,0.763407
5,2.291300,0.947129,0.763407


TrainOutput(global_step=6790, training_loss=1.2581720078777898, metrics={'train_runtime': 1782.1202, 'train_samples_per_second': 3.81, 'train_steps_per_second': 3.81, 'total_flos': 3551229326131200.0, 'train_loss': 1.2581720078777898, 'epoch': 5.0})

In [19]:
print("Evaluating after training...")
print("Post-training evaluation results:", trainer.evaluate())

Evaluating after training...


Post-training evaluation results: {'eval_loss': 0.9471291303634644, 'eval_accuracy': 0.7634069400630915, 'eval_runtime': 57.4061, 'eval_samples_per_second': 11.044, 'eval_steps_per_second': 11.044, 'epoch': 5.0}


In [20]:
from datetime import date
filename = f"{model_name}.-LORA.-{date.today()}"
# trainer.save_model(filename)

In [21]:
peft_model.save_pretrained(filename)

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [22]:
del ds
del peft_model
del trainer
torch.cuda.empty_cache()

In [23]:
from peft import AutoPeftModelForSequenceClassification
# model = AutoPeftModelForSequenceClassification.from_pretrained(filename)

NUM_LABELS = len(names)
lora_model = AutoPeftModelForSequenceClassification.from_pretrained(filename, num_labels=NUM_LABELS,\
                                                                    ignore_mismatched_sizes=True).to(device)

if model_name == 'gpt2':
    lora_model.config.pad_token_id =  tokenizer.pad_token_id


attention_mask = tokenized_dataset['test']['attention_mask'][:30].to(device)
input_ids = tokenized_dataset['test']['input_ids'][:30].to(device)
labels = tokenized_dataset['test']['label'][:30].to(device)
with torch.no_grad():
    output = lora_model(input_ids, attention_mask=attention_mask)

torch.mean(output.logits.argmax(dim=1).eq(labels).to(torch.float32)).item()

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0.7333333492279053

## Quantization

In [25]:
model_name = 'gpt2'

if model_name == 'gpt2':
    target_modules=["c_proj"]
elif model_name == 'bert-base-uncased':
    target_modules=["query","key","value"]

In [26]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments
from peft import prepare_model_for_kbit_training, LoftQConfig



model = AutoModelForSequenceClassification.from_pretrained(model_name,
    id2label=id2label,
    label2id=label2id,
    num_labels=len(names),
    load_in_4bit=True,

    device_map="auto")

if model_name == 'gpt2':
    model.config.pad_token_id =  tokenizer.pad_token_id
    
    


from peft import LoraConfig, get_peft_model
loftq_config = LoftQConfig(loftq_bits=4)
config = LoraConfig(

    r=1, 
    lora_alpha=32, 
    target_modules = target_modules, 
    lora_dropout=0.05, 
    bias="none", 
    task_type="SEQ_CLS"
)


model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model,8)
model = get_peft_model(model, config)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
epochs = 2

In [28]:
args = TrainingArguments(
        per_device_train_batch_size= per_device_train_batch_size,
        per_device_eval_batch_size = per_device_train_batch_size,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy='epoch',
        evaluation_strategy="epoch",
        load_best_model_at_end=True,
        num_train_epochs=epochs,

    metric_for_best_model='accuracy',

    )


trainer = Trainer(
    model=model,
    train_dataset = tokenized_dataset['train'], 
    eval_dataset =  tokenized_dataset['val'], 
    args= args,
    data_collator= DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics
)



In [29]:
print("Evaluating before training...")
print("Pre-training evaluation results:", trainer.evaluate())

/opt/conda/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


Evaluating before training...


Pre-training evaluation results: {'eval_loss': 3.9301881790161133, 'eval_accuracy': 0.250788643533123, 'eval_runtime': 40.997, 'eval_samples_per_second': 15.465, 'eval_steps_per_second': 15.465}


In [30]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.000000,0.437165,0.856467
2,0.000000,0.400139,0.949527


TrainOutput(global_step=2716, training_loss=0.8303643656409015, metrics={'train_runtime': 724.1975, 'train_samples_per_second': 3.75, 'train_steps_per_second': 3.75, 'total_flos': 711835011514368.0, 'train_loss': 0.8303643656409015, 'epoch': 2.0})

In [31]:
print("Evaluating after training...")
print("Post-training evaluation results:", trainer.evaluate())

Evaluating after training...


Post-training evaluation results: {'eval_loss': 0.40013912320137024, 'eval_accuracy': 0.9495268138801262, 'eval_runtime': 40.0478, 'eval_samples_per_second': 15.831, 'eval_steps_per_second': 15.831, 'epoch': 2.0}


In [32]:
predict,label_ids,metrics = trainer.predict(tokenized_dataset['test'])
metrics

{'test_loss': 0.47817572951316833,
 'test_accuracy': 0.9227941176470589,
 'test_runtime': 17.6046,
 'test_samples_per_second': 15.451,
 'test_steps_per_second': 15.451}